## Лабораторная работа номер 1 
### Работу выполнил: Татур Стефан




Задание 1:

Спроектируйте и реализуйте базу данных

1)студентов (фио. др. и номер студбилета)

2)группы(назвние)

3)предметы (название, к-во часов) которые они изучают

4)вид контроля (экзамен, зачет-рейтинг и кр)

5)оценка по предмету с датой проставления полученная конкретным студентом.

In [7]:
import sqlite3 as sq

In [9]:
con = sq.connect('university.db')

with con:
  cur = con.cursor() #вызов СУБД
  cur.execute(''' CREATE TABLE IF NOT EXISTS students(
              id INTEGER PRIMARY KEY AUTOINCREMENT,
              full_name TEXT NOT NULL,
              birth_date DATE NOT NULL,
              id_student TEXT UNIQUE NOT NULL)
              ''')

cur.execute("""
    CREATE TABLE IF NOT EXISTS groups (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT UNIQUE NOT NULL  )
    """)

cur.execute(''' CREATE TABLE IF NOT EXISTS subjects(
                  id INTEGER PRIMARY KEY AUTOINCREMENT,
                  sub_name TEXT NOT NULL,
                  hours INTEGER NOT NULL)
                  ''')

cur.execute(''' CREATE TABLE IF NOT EXISTS control_type(
              id INTEGER PRIMARY KEY AUTOINCREMENT,
              type TEXT UNIQUE NOT NULL)
              ''')

cur.execute(''' CREATE TABLE IF NOT EXISTS grades (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                student_id INTEGER,
                subject_id INTEGER,
                control_type_id INTEGER,
                grade INTEGER,
                date DATE,
                FOREIGN KEY (student_id) REFERENCES students(id),
                FOREIGN KEY (subject_id) REFERENCES subjects(id),
                FOREIGN KEY (control_type_id) REFERENCES control_type(id))
            ''')

students_data = [ ("Егор Егорович Егоров", "2001-01-01", "123436"),("Олег Олегович Олегов","2001-02-04","435345")]
group_data = [("Группа 2",),("Группа 3",)]
subjects_data = [("Линал", 60),("Дифференциальная геометрия",40)]
control_data = [("Тест",),("Коллоквиум",)]
grades_data = [(2, 1, 2, 5, "2024-01-15"),(2,2,2,4,"2024-01-16")]

with con:
    cur = con.cursor()

    # Вставка студентов
    cur.executemany("INSERT INTO students (full_name, birth_date, id_student) VALUES (?, ?, ?)",
                students_data)

    # Вставка группы
    cur.executemany("INSERT INTO groups (name) VALUES (?)", group_data)

    # Вставка предметов
    cur.executemany("INSERT INTO subjects (sub_name, hours) VALUES (?, ?)", subjects_data)

    # Вставка видов контроля
    cur.executemany("INSERT INTO control_type (type) VALUES (?)", control_data )

    # Вставка оценок
    cur.executemany("INSERT INTO grades (student_id, subject_id, control_type_id, grade, date) VALUES (?, ?, ?,?, ?)",
                grades_data)

# Закрытие соединения с базой данных
con.close()


In [11]:
con = sq.connect('university.db')

with con:
    cur = con.cursor()

    # Вывод студентов
    print("Студенты:")
    cur.execute("SELECT * FROM students")
    students = cur.fetchall()
    for student in students:
        print(student)

    # Вывод групп
    print("\nГруппы:")
    cur.execute("SELECT * FROM groups")
    groups = cur.fetchall()
    for group in groups:
        print(group)

    # Вывод предметов
    print("\nПредметы:")
    cur.execute("SELECT * FROM subjects")
    subjects = cur.fetchall()
    for subject in subjects:
        print(subject)

    # Вывод видов контроля
    print("\nВиды контроля:")
    cur.execute("SELECT * FROM control_type")
    control_types = cur.fetchall()
    for control in control_types:
        print(control)

    # Вывод оценок
    print("\nОценки:")
    cur.execute("SELECT * FROM grades")
    grades = cur.fetchall()
    for grade in grades:
        print(grade)

# Закрытие соединения с базой данных
con.close()

Студенты:
(1, 'Егор Егорович Егоров', '2001-01-01', '123436')
(2, 'Олег Олегович Олегов', '2001-02-04', '435345')

Группы:
(1, 'Группа 2')
(2, 'Группа 3')

Предметы:
(1, 'Линал', 60)
(2, 'Дифференциальная геометрия', 40)

Виды контроля:
(1, 'Тест')
(2, 'Коллоквиум')

Оценки:
(1, 2, 1, 2, 5, '2024-01-15')
(2, 2, 2, 2, 4, '2024-01-16')


Задание 2:

создайте таблицы связей между этими 5ю таблицами
Заполните эти таблицы

In [19]:
con = sq.connect('university.db')

with con:
  cur = con.cursor() #вызов СУБД
  cur.execute(''' CREATE TABLE IF NOT EXISTS student_groups (
    student_id INTEGER,
    course_id INTEGER,

    FOREIGN KEY (student_id) REFERENCES students(id),
    FOREIGN KEY (course_id) REFERENCES groups(id),
    PRIMARY KEY (student_id, course_id)
);
''')

  cur.execute(''' CREATE TABLE IF NOT EXISTS subjects_contol_type (
    subjects_id INTEGER,
    type_id INTEGER,

    FOREIGN KEY (subjects_id) REFERENCES subjects(id),
    FOREIGN KEY (type_id) REFERENCES groups(id),
    PRIMARY KEY (subjects_id, type_id)
);
''')


  cur.execute(''' CREATE TABLE IF NOT EXISTS student_grades (
    student_id INTEGER,
    grade_id INTEGER,

    FOREIGN KEY (student_id) REFERENCES student(id),
    FOREIGN KEY (grade_id ) REFERENCES grades(id),
    PRIMARY KEY (student_id, grade_id)
);
''')

  cur.execute("INSERT OR IGNORE INTO student_groups (student_id, course_id) VALUES (?, ?)", (1, 1))  # Иванов - Группа 1
  cur.execute("INSERT OR IGNORE INTO student_groups (student_id, course_id) VALUES (?, ?)", (2, 3))  # Егоров - Группа 3
  cur.execute("INSERT OR IGNORE INTO student_groups (student_id, course_id) VALUES (?, ?)", (3, 1))  # Олегов - Группа 1


  cur.execute("INSERT OR IGNORE INTO subjects_contol_type (subjects_id, type_id) VALUES (?, ?)", (1, 3))  # Матемамтика - Коллоквиум
  cur.execute("INSERT OR IGNORE INTO subjects_contol_type (subjects_id, type_id) VALUES (?, ?)", (2, 1))  # Линал - Экзамен
  cur.execute("INSERT OR IGNORE INTO subjects_contol_type (subjects_id, type_id) VALUES (?, ?)", (3, 2))  # Дифференциальная геометрия - Тест

  cur.execute("INSERT OR IGNORE INTO student_grades (student_id, grade_id) VALUES (?, ?)", (1, 3))  # Иванов - четверка
  cur.execute("INSERT OR IGNORE INTO student_grades (student_id, grade_id) VALUES (?, ?)", (2, 1))  # Егоров - пятерка
  cur.execute("INSERT OR IGNORE INTO student_grades (student_id, grade_id) VALUES (?, ?)", (3, 2))  # Олегов - пятерка

# Закрытие соединения с базой данных
con.close()

Задание 3:

Напишите запрос, который покзыает самого (или самых, если их несколько) полохи студентов каждой группе

In [22]:
con = sq.connect('university.db')

with con:
    cur = con.cursor()

    # SQL query to find the worst student in group 1
    query = '''
    SELECT s.full_name AS student_name, MIN(gr.grade) AS lowest_grade
    FROM students s
    JOIN student_groups sg ON s.id = sg.student_id
    JOIN grades gr ON s.id = gr.student_id
    WHERE sg.course_id = 1
    GROUP BY s.id
    HAVING MIN(gr.grade) = (
        SELECT MIN(gr2.grade)
        FROM grades gr2
        JOIN student_groups sg2 ON gr2.student_id = sg2.student_id
        WHERE sg2.course_id = 1
    );
    '''

    cur.execute(query)

    results = cur.fetchall()

    if results:
        for row in results:
            print(f"Student: {row[0]}, Самая низкая оценка: {row[1]}")
    else:
        print("В группе 1 нет студентов.")

con.close()


В группе 1 нет студентов.


Задание 4: Создайте бекап этой базы данных

In [24]:
con = sq.connect('university.db')

with con:
    with open("sql_dump.sql", "w") as f:
        for sql in con.iterdump():
            f.write(sql + "\n")

con.close()



Задание 5:

Удалите все таблици и восстановите их из бекапа


In [ ]:
con = sq.connect('university.db')

with con:
  cur = con.cursor()

  tables = [
      "students",
      "groups",
      "subjects",
      "control_type",
      "grades",
      "student_groups",
      "subjects_control_type",
      "student_grades"
  ]
  for table in tables:
    cur.execute(f"DROP TABLE IF EXISTS {table};")
    print(f"Таблица {table} удалена\n")

con.close()